In [ ]:
!pip install flask
!pip install flask-ngrok

In [9]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, flash, request, redirect
from werkzeug.utils import secure_filename
import pathlib  #to check the path and verify if the required SW is allready installed
import os
#import magic
import urllib.request

app = Flask(__name__)
run_with_ngrok(app)
UPLOAD_FOLDER = '/content/Project/Input'
!mkdir -v $UPLOAD_FOLDER
app.secret_key = "HWI"
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024
 
ALLOWED_EXTENSIONS = set(['png', 'jpg', 'jpeg'])

def makeModel():
 print("\nLaunching 3D-Reconstruction application on Google colab\n")
 # Folder paths
 currentDir = "/content/Project" #work directory
 inputFiles = currentDir + "/Input" # folder in work directory with the input images
 outputFiles = currentDir + "/output" # folder in work directory with output files
 meshroomFolder = currentDir + "/meshroom" # folder in work directory with all meshroom files

 # Creating all directories
 !mkdir -v $currentDir # will be skipped if allready existing
 !mkdir -v $outputFiles #temporary folder in work directory to store 3D output files. Should be deleted once 3D output is moved to google drive folder.
 !mkdir -v $meshroomFolder # will be skipped if allready existing

 # Go to work directory
 %cd $currentDir
 # Check if meshroom is allready installed. If not, download and install meshroom
 meshroom_files = pathlib.Path("/content/Project/meshroom/Meshroom-2019.2.0")
 if not meshroom_files.exists ():
  print("\nMeshroom not found.\nInstalling Meshroom...")
  !wget -N https://github.com/alicevision/meshroom/releases/download/v2019.2.0/Meshroom-2019.2.0-linux.tar.gz
  !tar xzf Meshroom-2019.2.0-linux.tar.gz -C $meshroomFolder
  %mv -v /content/Project/meshroom/Meshroom-2019.2.0/* $meshroomFolder
 else :
  print ("\nMeshroom is allredy installed. Skipping a new installation...")
 # Execute Meshroom
 startMeshroom = meshroomFolder+"/meshroom_photogrammetry"
 !$startMeshroom --input $inputFiles --output $outputFiles
 print('###################################################################################################')

def allowed_file(filename):
  return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS
  
@app.route('/')
def upload_form():
  return render_template('upload.html')
 
@app.route('/', methods=['POST'])
def upload_file():
 if request.method == 'POST':
        # check if the post request has the files part
  if 'files[]' not in request.files:
   flash('No file part')
   return redirect(request.url)
  files = request.files.getlist('files[]')
  for file in files:
   if file and allowed_file(file.filename):
    filename = secure_filename(file.filename)
    file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
  flash('File(s) successfully uploaded')
  return redirect('/')
  makeModel()

app.run()

mkdir: cannot create directory ‘/content/Project/Input’: File exists
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://ea85-34-68-199-111.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


[2021-08-22 05:35:34,957] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.7/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-9-1ca696ef2f6b>", line 54, in upload_form
    return render_template('upload.html')
  File 

In [ ]:
import shutil
dir = '/content/Project/Input'
shutil.rmtree(dir)